In [15]:
import http.client, urllib.parse

host = "pwnable.kr"
header = {
    "Cookie": "PHPSESSID=i77tu2mn5vmtjnj5ft6gd9cn24",
}

In [5]:
method = "GET"
path ="/play.php"
body = None
conn = http.client.HTTPSConnection(host)
conn.request(method, path, body, header)
resp = conn.getresponse()
resp_header = resp.getheaders()
resp_body = resp.read()
url = "https://" + host + path
print("HTTP Request: {} -> {} {} bytes".format(url, resp.status, len(resp_body)))
conn.close()

HTTP Request: https://pwnable.kr/play.php -> 200 40619 bytes


In [7]:
import re
data = resp_body.decode()
match = re.findall("(<figure>.+?</figure>)", data)
print(len(match))

68


In [36]:
tasks = dict()
for item in match:
    task = dict()
    key = re.findall("<figcaption>(.+?)</figcaption>", item)[0]
    task["name"] = key
    task["id"] = re.findall("onLayer\((.+?)\)", item)[0]
    task["url"] = "/playproc.php?no={}".format(task["id"])
    task["img_url"] = re.findall("src=\'(.+?)\'", item)[0].replace(" ", "%20")
    tasks[key] = task

In [47]:
method = "GET"
    
for name in tasks.keys():
    path ="/playproc.php"
    query = {"no": tasks[name]["id"]}
    path += "?" + urllib.parse.urlencode(query)
    body = None
    conn = http.client.HTTPSConnection(host)
    conn.request(method, path, body, header)
    resp = conn.getresponse()
    resp_header = resp.getheaders()
    resp_body = resp.read()
    url = "https://" + host + path
    print("HTTP Request: {} -> {} {} bytes".format(url, resp.status, len(resp_body)))
    conn.close()
    data = resp_body.decode()
    tasks[name]["points"] = re.findall("\s*([0-9]+?)\s*pt", data)[0]
    tasks[name]["solved"] = re.findall("pwned \((.+?)\)", data)[0]
    tasks[name]["description"] = re.findall("<textarea.+?>([\s\S]+?)</textarea>", data)[0]
    tasks[name]["writup_url"] = re.findall("<a href=\'(.+?)\'", data)[0]

HTTP Request: https://pwnable.kr/playproc.php?no=25 -> 200 1241 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=70 -> 200 1194 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=33 -> 200 1258 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=1 -> 200 1305 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=68 -> 200 1357 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=45 -> 200 1307 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=72 -> 200 1483 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=52 -> 200 1360 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=35 -> 200 1371 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=47 -> 200 1320 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=41 -> 200 1245 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=65 -> 200 1218 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=34 -> 200 1189 bytes
HTTP Request: https://pwnable.kr/playproc.php?no=48 -> 200 1198 bytes
HTTP Request: https:/

In [59]:
names = tasks.keys()
names = sorted(names, key=lambda name: int(tasks[name]["solved"]), reverse=True)

In [60]:
for name in names:
    print("="*120)
    print("{id:>4}\t\033[1;31m{name:24}\033[0m {points:>8} {solved:>8}".format(**tasks[name]))
    print("-"*120)
    print("{description}".format(**tasks[name]))

   1	fd                              1    23293
------------------------------------------------------------------------------------------------------------------------
Mommy! what is a file descriptor in Linux?

* try to play the wargame your self but if you are ABSOLUTE beginner, follow this tutorial link:
https://youtu.be/971eZhMHQQw

ssh fd@pwnable.kr -p2222 (pw:guest)
   2	collision                       3    13951
------------------------------------------------------------------------------------------------------------------------
Daddy told me about cool MD5 hash collision today.
I wanna do something like that too!

ssh col@pwnable.kr -p2222 (pw:guest)
   3	bof                             5    10055
------------------------------------------------------------------------------------------------------------------------
Nana told me that buffer overflow is one of the most common software vulnerability. 
Is that true?

Download : http://pwnable.kr/bin/bof
Download : http://pwnabl

In [62]:
import json
file_path = "./tasks.json"
with open(file_path, "wt") as f:
    f.write(json.dumps(tasks))

### submit

In [64]:
import http.client, urllib.parse

session_id = "i77tu2mn5vmtjnj5ft6gd9cn24"
flag = "daddy, I just pwned a buFFer :)"

host = "pwnable.kr"
header = {
    "Cookie": "PHPSESSID=" + session_id,
    "Content-Type": "application/x-www-form-urlencoded",
}

method = "POST"
path ="/lib.php?cmd=auth"
param = {"flag": flag}
body = urllib.parse.urlencode(param)
conn = http.client.HTTPSConnection(host)
conn.request(method, path, body, header)
resp = conn.getresponse()
resp_header = resp.getheaders()
resp_body = resp.read()
url = "https://" + host + path
print("HTTP Request: {} -> {} {} bytes".format(url, resp.status, len(resp_body)))
conn.close()
print(resp_body.decode())

HTTP Request: https://pwnable.kr/lib.php?cmd=auth -> 200 94 bytes
<script>alert("You already authenticated this flag");window.location.href='play.php';</script>
